In [1]:
import pandas as pd 
import numpy as np

In [157]:
train = pd.read_csv('../data/v2/training_v2.csv')
test = pd.read_csv('../data/v2/test_v2.csv')

In [158]:
# train = train[train['segment']==1].reset_index(drop=True)
train = train[train['segment']==2].reset_index(drop=True)
# test = test[test['segment']==1].reset_index(drop=True)
test = test[test['segment']==2].reset_index(drop=True)

In [159]:
test.drop_duplicates(subset=['id'], inplace=True)

In [160]:
train.Type.fillna(value="Nope", inplace=True)
test.Type.fillna(value="Nope", inplace=True)

In [161]:
train.head()

,application_date,segment,case_count,day,year,month,Holiday,Type,cos_day,sin_day,cos_mon,sin_mon
0,2017-04-01,2,0.0,1,2017,4,NaN,Nope,0.978148,0.207912,-0.5,0.866025
1,2017-04-02,2,0.0,2,2017,4,NaN,Nope,0.913545,0.406737,-0.5,0.866025
2,2017-04-03,2,0.0,3,2017,4,NaN,Nope,0.809017,0.587785,-0.5,0.866025
3,2017-04-04,2,0.0,4,2017,4,Rama Navami,G,0.669131,0.743145,-0.5,0.866025
4,2017-04-05,2,0.0,5,2017,4,NaN,Nope,0.500000,0.866025,-0.5,0.866025


In [162]:
test.head()

,id,application_date,segment,day,year,month,Holiday,Type,cos_mon,sin_mon,cos_day,sin_day
0,88,2019-07-24,2,24,2019,7,NaN,Nope,-0.866025,-0.5,0.151428,-0.988468
1,89,2019-07-25,2,25,2019,7,NaN,Nope,-0.866025,-0.5,0.347305,-0.937752
2,90,2019-07-26,2,26,2019,7,NaN,Nope,-0.866025,-0.5,0.528964,-0.848644
3,91,2019-07-27,2,27,2019,7,NaN,Nope,-0.866025,-0.5,0.688967,-0.724793
4,92,2019-07-28,2,28,2019,7,NaN,Nope,-0.866025,-0.5,0.820763,-0.571268


# Encoding Type of Holiday

In [163]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [164]:
la = LabelEncoder()
on = OneHotEncoder()

In [165]:
train.Type.unique()

array(['Nope', 'G', 'O', 'R', 'C', 'S', 'M'], dtype=object)

In [166]:
train['Type_Mat'] = la.fit_transform(train['Type'])

In [167]:
type_mat = on.fit_transform(train['Type_Mat'].values.reshape(-1,1))

In [168]:
mat = type_mat.todense()

In [169]:
mat.shape

(13840, 7)

In [170]:
mat = np.append(mat, train['segment'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['month'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['year'].values.reshape(-1, 1), axis=1)
# mat = np.append(mat, train['sin_mon'].values.reshape(-1, 1), axis=1)

In [171]:
mat.shape

(13840, 11)

In [172]:
test.head()

,id,application_date,segment,day,year,month,Holiday,Type,cos_mon,sin_mon,cos_day,sin_day
0,88,2019-07-24,2,24,2019,7,NaN,Nope,-0.866025,-0.5,0.151428,-0.988468
1,89,2019-07-25,2,25,2019,7,NaN,Nope,-0.866025,-0.5,0.347305,-0.937752
2,90,2019-07-26,2,26,2019,7,NaN,Nope,-0.866025,-0.5,0.528964,-0.848644
3,91,2019-07-27,2,27,2019,7,NaN,Nope,-0.866025,-0.5,0.688967,-0.724793
4,92,2019-07-28,2,28,2019,7,NaN,Nope,-0.866025,-0.5,0.820763,-0.571268


In [173]:
test['Type_Mat'] = la.transform(test['Type'])
mat_test = on.transform(test['Type_Mat'].values.reshape(-1,1))

In [174]:
mat_test = mat_test.todense()

In [175]:
mat_test = np.append(mat_test, test['segment'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['day'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['month'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['year'].values.reshape(-1, 1), axis=1)
# mat_test = np.append(mat_test, test['sin_mon'].values.reshape(-1, 1), axis=1)

In [176]:
mat_test.shape

(93, 11)

In [177]:
from sklearn.linear_model import LinearRegression, HuberRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
# import lightgbm as lgb

In [178]:
X_train, X_test, y_train, y_test = train_test_split(mat, train['case_count'], test_size=0.33, random_state=42)

In [179]:
lr = LinearRegression()
hu = HuberRegressor()
sgd = SGDRegressor()

In [180]:
rf = RandomForestRegressor(n_estimators=100)
et = ExtraTreesRegressor(n_estimators=100)

In [181]:
rf.fit(X_train, y_train)
et.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [182]:
lr.fit(X_train, y_train)
hu.fit(X_train, y_train)
sgd.fit(X_train, y_train)

c:\programs files\python\python36\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [183]:
lr_pred = np.round(abs(lr.predict(X_test)))
hu_pred = np.round(abs(hu.predict(X_test)))
sgd_pred = np.round(abs(sgd.predict(X_test)))

In [184]:
rf_pred = np.round(abs(rf.predict(X_test)))
et_pred = np.round(abs(et.predict(X_test)))

In [185]:
mean_absolute_error(y_test, lr_pred) * 100/len(y_test), mean_absolute_error(y_test, hu_pred) * 100/len(y_test), mean_absolute_error(y_test, sgd_pred) * 100/len(y_test)

(21.491483241371487, 18.422683036795988, 28245486022389.41)

In [186]:
mean_absolute_error(y_test, rf_pred) * (100/len(y_test)), mean_absolute_error(y_test, et_pred) * (100/len(y_test))

(20.864817423268853, 20.87268644894354)

In [187]:
et_pred, rf_pred

(array([ 84., 706., 760., ...,  58., 967., 324.]),
 array([  86.,  752.,  734., ...,   58., 1005.,  322.]))

In [188]:
test_pred = np.round(hu.predict(mat_test))

In [189]:
test['case_count'] = test_pred.reshape(-1,1)

In [190]:
test[['id','application_date','segment','case_count']].to_csv('pred_baseline_rf_seg2.csv', index=False)

# LGM

In [85]:
median_val = np.median(train['case_count'].values)

In [89]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmsle',
    'max_depth': 6, 
    'learning_rate': 0.1,
    'verbose': 0}
#     'early_stopping_round': 20}
n_estimators = 100

In [124]:
n_iters = 5
preds_buf = []
err_buf = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(np.array(mat), train['case_count'].values.reshape(-1,1).ravel(), test_size=0.1, random_state=i)
    d_train = lgb.Dataset(x_train, label=y_train)
    d_valid = lgb.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

    preds = model.predict(x_valid)
#     preds = np.exp(preds)
    preds[preds < 0] = median_val
    err = mean_absolute_error(y_valid, preds) * 100/len(y_valid)
    err_buf.append(err)
    print('MAE% = ' + str(err))
    
    preds = model.predict(np.array(mat_test))
#     preds = np.exp(preds)
    preds[preds < 0] = median_val
    preds_buf.append(preds)

print('Mean MAE% = ' + str(np.mean(err_buf)) + ' +/- ' + str(np.std(err_buf)))
# Average predictions
preds = np.round(np.mean(preds_buf, axis=0))

MAE% = 2.133783523513568
MAE% = 2.226129904020018
MAE% = 2.161169430293224
MAE% = 2.1387945927443752
MAE% = 2.3745359280677167
Mean MAE% = 2.2068826757277806 +/- 0.09005247661674118


In [96]:
len(train['case_count'].values.reshape(-1,1).ravel())

82398

In [121]:
mat_test

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.93775213,
         -0.8660254 , -0.5       ],
        [ 0.        ,  0.        ,  0.        , ...,  0.98846832,
         -0.8660254 , -0.5       ],
        [ 0.        ,  0.        ,  0.        , ...,  0.99871651,
         -0.8660254 , -0.5       ],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.96807712,
          0.5       , -0.8660254 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.99871651,
          0.5       , -0.8660254 ],
        [ 0.        ,  0.        ,  0.        , ..., -0.98846832,
          0.5       , -0.8660254 ]])

In [128]:
test['case_count_lgb'] = preds

In [134]:
test = test.drop_duplicates(subset=['id'])

In [135]:
test[['id','application_date','segment','case_count_lgb']].to_csv('pred_baseline_lgb.csv', index=False)